In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    '''값이 1,2개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()<3 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

# (1) model1 -> 0 

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)


# 강한 추세성보이는 칼럼 중 일부 제거
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)

'''
trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)

trainA_31_x,testA_31_x = log(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = log(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = log(train_O_31_x,test_O_31_x)
'''

from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=6, encode='ordinal', strategy="quantile", random_state=1414)

features = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[features])
binned_df_train = enc.transform(train_T_31_x[features])
binned_df_test = enc.transform(test_T_31_x[features])
for i, feature in enumerate(features):
    train_T_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{feature}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=22, encode='ordinal', strategy="quantile", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 3461) (67, 3461) (343, 1001) (239, 1001)


In [ ]:
'''
10% -> 26,7
20% -> 28,5
2 -> 33,4
3 -> 26,5 (4,5)(21,0)
5;10 -> 25,6
2,48 - > 29, 4
'''

'''
17-> 3
9 -> 4
'''

In [4]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.6442300396904909 
 -0.6441937649548493 
 -0.6257823004654807 
 -0.6255805101477584 



In [4]:
model2 = CatBoostRegressor(random_state=2000,
                           depth=6,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=1000,
                           learning_rate=0.1)

In [5]:
model3 = CatBoostRegressor(random_state=4567,
                           depth=5,
                           verbose=500,
                           iterations=8888,
                           learning_rate=0.008)

In [6]:
model2.fit(trainA_31_x, trainA_31_y)
pred_a = model2.predict(testA_31_x)

0:	learn: 0.0179190	total: 87.3ms	remaining: 1m 27s
500:	learn: 0.0000163	total: 10.3s	remaining: 10.2s
999:	learn: 0.0000000	total: 20.2s	remaining: 0us


In [7]:
model3.fit(train_T_31_x, train_T_31_y)
pred_t = model3.predict(test_T_31_x)

0:	learn: 0.0089598	total: 6.26ms	remaining: 55.7s
500:	learn: 0.0056280	total: 2.21s	remaining: 36.9s
1000:	learn: 0.0037699	total: 4.4s	remaining: 34.6s
1500:	learn: 0.0023050	total: 6.6s	remaining: 32.5s
2000:	learn: 0.0014897	total: 8.8s	remaining: 30.3s
2500:	learn: 0.0009720	total: 11s	remaining: 28.1s
3000:	learn: 0.0006547	total: 13.2s	remaining: 25.9s
3500:	learn: 0.0004400	total: 15.4s	remaining: 23.8s
4000:	learn: 0.0002884	total: 17.7s	remaining: 21.6s
4500:	learn: 0.0001935	total: 20.1s	remaining: 19.6s
5000:	learn: 0.0001347	total: 22.5s	remaining: 17.5s
5500:	learn: 0.0000951	total: 24.9s	remaining: 15.3s
6000:	learn: 0.0000649	total: 27.5s	remaining: 13.2s
6500:	learn: 0.0000445	total: 30s	remaining: 11s
7000:	learn: 0.0000307	total: 32.5s	remaining: 8.75s
7500:	learn: 0.0000209	total: 34.9s	remaining: 6.46s
8000:	learn: 0.0000142	total: 37.3s	remaining: 4.13s
8500:	learn: 0.0000097	total: 39.8s	remaining: 1.81s
8887:	learn: 0.0000072	total: 41.8s	remaining: 0us


In [8]:
model3.fit(train_O_31_x, train_O_31_y)
pred_o = model3.predict(test_O_31_x)

0:	learn: 0.0060169	total: 1.26ms	remaining: 11.2s
500:	learn: 0.0020878	total: 441ms	remaining: 7.38s
1000:	learn: 0.0007660	total: 862ms	remaining: 6.79s
1500:	learn: 0.0002815	total: 1.31s	remaining: 6.45s
2000:	learn: 0.0001035	total: 1.75s	remaining: 6.04s
2500:	learn: 0.0000380	total: 2.25s	remaining: 5.75s
3000:	learn: 0.0000140	total: 2.75s	remaining: 5.38s
3500:	learn: 0.0000051	total: 3.2s	remaining: 4.93s
4000:	learn: 0.0000019	total: 3.67s	remaining: 4.49s
4500:	learn: 0.0000007	total: 4.12s	remaining: 4.01s
5000:	learn: 0.0000003	total: 4.56s	remaining: 3.54s
5500:	learn: 0.0000001	total: 5.01s	remaining: 3.09s
6000:	learn: 0.0000000	total: 5.47s	remaining: 2.63s
6500:	learn: 0.0000000	total: 5.92s	remaining: 2.17s
7000:	learn: 0.0000000	total: 6.4s	remaining: 1.73s
7500:	learn: 0.0000000	total: 6.88s	remaining: 1.27s
8000:	learn: 0.0000000	total: 7.36s	remaining: 816ms
8500:	learn: 0.0000000	total: 7.86s	remaining: 358ms
8887:	learn: 0.0000000	total: 8.26s	remaining: 0us


## 0

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

In [46]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x+5))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)


# 강한 추세성보이는 칼럼 중 일부 제거
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)

'''
trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)

trainA_31_x,testA_31_x = log(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = log(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = log(train_O_31_x,test_O_31_x)
'''

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2107) (67, 2107) (343, 668) (239, 668)


In [70]:
model1 = CatBoostRegressor(random_state=1333,
                           depth=5,
                           verbose=500,
                           iterations=2500,
                           learning_rate=0.055)

In [48]:
model4 = CatBoostRegressor(random_state=1333,
                           depth=6,
                           verbose=500,
                           iterations=500,
                           learning_rate=0.1283)

In [71]:
model1.fit(trainA_31_x, trainA_31_y)
pred_a0 = model1.predict(testA_31_x)

0:	learn: 0.0017728	total: 15ms	remaining: 37.5s
500:	learn: 0.0000511	total: 5.19s	remaining: 20.7s
1000:	learn: 0.0000023	total: 10.5s	remaining: 15.7s
1500:	learn: 0.0000001	total: 15.8s	remaining: 10.5s
2000:	learn: 0.0000000	total: 21.3s	remaining: 5.32s
2499:	learn: 0.0000000	total: 26.9s	remaining: 0us


In [72]:
model1.fit(train_T_31_x, train_T_31_y)
pred_t0 = model1.predict(test_T_31_x)

0:	learn: 0.0008522	total: 7.08ms	remaining: 17.7s
500:	learn: 0.0000436	total: 2.11s	remaining: 8.43s
1000:	learn: 0.0000029	total: 4.22s	remaining: 6.32s
1500:	learn: 0.0000003	total: 6.34s	remaining: 4.22s
2000:	learn: 0.0000000	total: 8.46s	remaining: 2.11s
2499:	learn: 0.0000000	total: 10.6s	remaining: 0us


In [73]:
testA_31['Y_2'] = pred_a
test_T_31['Y_2'] = pred_t
testA_31['Y_0'] = pred_a0
test_T_31['Y_0'] = pred_t0
test_O_31['Y'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_0']<1.709298761129632), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_2']>-0.6256814053066195), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_0']<1.709298761129632), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_2']>-0.6256814053066195), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y']<-0.64421190232267), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y']>-0.6256814053066195), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('0225.csv',index=False)

In [74]:
pp = pd.read_csv('0225.csv')

In [75]:
pp['Y_Class'].value_counts()

1    263
0     37
2     10
Name: Y_Class, dtype: int64

In [76]:
pp[pp['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
15,TEST_015,0
35,TEST_035,0
37,TEST_037,0
39,TEST_039,0
41,TEST_041,0
42,TEST_042,0


In [38]:
st[st['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
7,TEST_007,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
35,TEST_035,0
36,TEST_036,0
37,TEST_037,0
39,TEST_039,0
41,TEST_041,0


In [78]:
pp[pp['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
109,TEST_109,2
115,TEST_115,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
177,TEST_177,2
183,TEST_183,2


In [77]:
st[st['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
137,TEST_137,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
179,TEST_179,2
183,TEST_183,2


In [32]:
st = pd.read_csv('따로.csv')